#### 2020-2 YBIGTA 교육세션
# ML 심화 과제

타이타닉 데이터를 활용한 과제입니다. 각 단계에서 추가적으로 필요한 라이브러리가 있다면 자유롭게 불러와서 사용해주세요.

과제할 때 꿀팁!

1. 이전 세션들에서의 자료를 찾아본다.
2. 질문방에 질문한다.
3. 구글링한다.
4. 복붙하지 않고 직접 타이핑해서 코드를 완성한다.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# csv 파일 열기
df = pd.read_csv('titanic2.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,0.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,1.0
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,0.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,0.0
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,0.0


In [4]:
# Parch는 함께 탑승한 부모 혹은 자녀의 수, SibSp는 함께 탑승한 형제나 배우자의 수입니다.
# 이 둘을 합쳐서 'Family' column을 만들어주세요.
df['Family'] = df['Parch'] + df['SibSp']

In [5]:
# 필요한 것들만 추리겠습니다. 'Sex', 'Age', 'Family', 'Fare', 'Embarked', 'Survived' 순으로
# 데이터 프레임을 만들어주세요.

df = df[['Sex', 'Age', 'Family', 'Fare', 'Embarked', 'Survived']]
df.head()

,Sex,Age,Family,Fare,Embarked,Survived
0,0,22.0,1,7.2500,0.0,0
1,1,38.0,1,71.2833,1.0,1
2,1,26.0,0,7.9250,0.0,1
3,1,35.0,1,53.1000,0.0,1
4,0,35.0,0,8.0500,0.0,0


In [6]:
# 데이터 프레임을 독립변수 X, 종속변수 y로 나눠주세요.
# 종속변수 y는 Survived, X는 나머지 칼럼들

X = df.drop(columns=['Survived'], inplace=False, axis=1)
y = df['Survived']

In [7]:
# train과 test를 70:30으로 나눠주는 패키지를 불러온 뒤 X와 y를 각각 나눠주세요.

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# X_train, X_test shape 확인

print(X_train.shape)
print(X_test.shape)

(712, 5)
(179, 5)


### 랜덤 포레스트

BaggingClassifier와 DecisionTreeClassifier로 랜덤포레스트를 직접 만들어보세요.

In [9]:
# 200개 트리를 만들고 각 트리마다 15개의 샘플을 넣습니다. 각 트리의 노드 개수는 31개로 하고, 모델명은 bagging이라고 합시다.
# 배깅이라는 걸 표시해주세요. cpu는 사용할 수 있는 모든 것을 씁니다.

from sklearn.ensemble import BaggingClassifier 
from sklearn.tree import DecisionTreeClassifier

bagging = BaggingClassifier(DecisionTreeClassifier(max_leaf_nodes = 31, random_state=42),
                            n_estimators = 200, max_samples = 15, bootstrap = True, n_jobs = -1
                           )

bagging.fit(X_train, y_train)

# train의 score와 test의 score를 출력합니다.

print('training set accuracy:', bagging.score(X_train, y_train))
print('test set accuracy:', bagging.score(X_test, y_test))

training set accuracy: 0.7921348314606742
test set accuracy: 0.7821229050279329


In [10]:
# 랜덤포레스트 패키지를 불러와 100개 트리를 만들고 노드 개수는 15개로 설정한 모델(모델명은 forest로 하겠습니다.)을 만듭니다.

from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators = 100, max_leaf_nodes = 15, n_jobs=-1)
forest.fit(X_train, y_train)

# train의 score와 test의 score를 출력합니다.

print('training set accuracy:', forest.score(X_train, y_train))
print('test set accuracy:', forest.score(X_test, y_test))

training set accuracy: 0.8441011235955056
test set accuracy: 0.8044692737430168


In [11]:
# 원본 데이터프레임 df의 features는 다음과 같습니다.

df.columns[:-1]

Index(['Sex', 'Age', 'Family', 'Fare', 'Embarked'], dtype='object')

In [15]:
# 위 정보를 이용해 forest 모델의 feature importance와 각 중요도가 가리키는 feature를 zip해서 출력해보세요.

for name, score in zip(df.columns[:-1], forest.feature_importances_):
    print(f'{name}  --->  {score:.3f}')

Sex  --->  0.520
Age  --->  0.107
Family  --->  0.122
Fare  --->  0.230
Embarked  --->  0.021


### 부스팅

교육세션에서 배운 모델이나 추가적으로 알고 있는 부스팅 모델 중 마음에 드는 2개를 골라서 주어진 데이터셋에 대해 각각 돌려주세요.
- 돌린 다음 각 모델의 training, test 성능을 확인해봅니다.
- 더 높은 성능을 낼 수 있는 파라미터를 찾아보는 시도를 해보아도 좋습니다!

In [20]:
# Model 1 : catboost
from catboost import CatBoostClassifier

cbst_clf = CatBoostClassifier()
cbst_clf.fit(X_train, y_train, verbose=False)
cbst_clf.score(X_test, y_test)

0.8100558659217877

In [22]:
# Model 2 : lgbm
from lightgbm import LGBMClassifier
lgbm_clf = LGBMClassifier()
lgbm_clf.fit(X_train, y_train)
lgbm_clf.score(X_test, y_test)

0.8491620111731844

아무 parameter tuning 없이 진행하였는데도 상당히 좋은 결과를 보여준다!!

lgbm이 다른 앙상블과 다른 점에 대해 적어주세요.

LGBM은 다른 모델과는 다르게, leaf-wise 방식을 채택하였다. <br>
트리를 균형 잡히게(level wise) 학습하는 다른 방식은 학습하는데 시간이 많이 걸리지만, LGBM은 트리가 균형 잡히지 않고 학습하므로 시간이 적게 걸린다. <br>
level wise 방식은 결국 균형 잡히게 발달하므로 시간이 오래 걸리는 대신 과적합을 대응할 수 있지만, LGBM은 깊이가 길어질 수 있어 과적합에 취약하다는 단점이 있다.

### 스태킹

스태킹이 다른 앙상블과 다른 점에 대해 적어주세요.

스태킹은 특히 보깅과 배깅을 이용하는 다른 앙상블 기법과 차이점을 보인다. <br>
보깅과 배깅은 결국 여러 학습기에서 나온 결과를 voting이라는 과정을 통해 최종 결과를 도출한다. <br>
이때 여러 결과 중 평균값이나 중간값, 최빈값 등을 사용하는 방식이 있다. <br><br>
반면에 스태킹은 여러 학습기에서 나온 결과를 다시 input data 삼아 새로운 학습을 하는 blender 모델을 학습한다. <br>
이 blender 모델이 학습하여 나온 결과를 최종 결과로 사용하는 방식이 바로 스태킹 방식이다.